In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from qiime2 import Artifact, Visualization

from skbio.stats.distance import mantel, permanova, DistanceMatrix
from scipy.spatial.distance import pdist, cdist

# Setup

In [41]:
wuni = Artifact.load("core-diversity-results/weighted_unifrac_distance_matrix.qza").view(DistanceMatrix)
uuni = Artifact.load("core-diversity-results/unweighted_unifrac_distance_matrix.qza").view(DistanceMatrix)
bray = Artifact.load("core-diversity-results/bray_curtis_distance_matrix.qza").view(DistanceMatrix)

In [42]:
metadata = pd.read_csv("data/metadata.tsv", sep="\t", index_col="Sampleid")

In [43]:
metadata.columns

Index(['Subjectcode', 'Timepoint', 'Sex', 'Length', 'Birth Weight',
       'Gestational Age', 'Infections', 'Antibiotic use', 'Mothers Age',
       'Mothers Height', 'Mothers Weight', 'Education', 'Ethnicity Race',
       'Zip code', 'Annual Household Income', 'Smoke',
       'Smoke during pregnancy', 'Smoke after delivery', 'Alcohol',
       'Drinks before pregnancy per week', 'Current H. pylori infection',
       'Pregnancy complications', 'Probiotics during pregnancy',
       'Post-natal probiotics', 'Post-natal multivitamins',
       'Antibiotic use during pregnancy', 'Antibiotic use which trimester',
       'Antibiotic use Post-natal', 'Other post-natal medications',
       'High cholesterol', 'EBM-FF-PBM', 'Maternal relevant medical hx',
       'Current feeding method', 'Solid foods given', 'Baby Antibiotic use',
       'Mother antibiotic use'],
      dtype='object')

# Categorical variables

! qiime diversity beta-group-significance \
--i-distance-matrix core-diversity-results/weighted_unifrac_distance_matrix.qza \
--m-metadata-file data/metadata.tsv \
--m-metadata-column Subjectcode \
--o-visualization core-diversity-results/PERMANOVA-subject.qzv

! qiime diversity beta-group-significance \
--i-distance-matrix core-diversity-results/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file data/metadata.tsv \
--m-metadata-column Subjectcode \
--o-visualization core-diversity-results/PERMANOVA-subject-uuni.qzv

! qiime diversity beta-group-significance \
--i-distance-matrix core-diversity-results/bray_curtis_distance_matrix.qza \
--m-metadata-file data/metadata.tsv \
--m-metadata-column Subjectcode \
--o-visualization core-diversity-results/PERMANOVA-subject-bray.qzv

### The variation between individuals is significantly larger than the variation within individuals

#### Weighted UniFrac
- pseudo-F = 2.834
- p-value = 0.001

#### Unweighted UniFrac
- test statistic = 1.59645
- p-value = 0.008

#### Bray-Curtis
- test statistic = 2.9183
- p-value = 0.001

In [44]:
t0 = metadata[metadata["Timepoint"]==0]
t1 = metadata[metadata["Timepoint"]==1]
t2 = metadata[metadata["Timepoint"]==2]

In [45]:
wuni.to_data_frame().loc[t0.index, t0.index]

Sampleid,B.F 1.1,B.F 2.1,B.F 3.1,B.F 4.1,B.F 5.1,B.F 6.1,B.F 7.1
Sampleid,,,,,,,
B.F 1.1,0.000000,1.028974,2.528423,1.686389,2.117143,0.334186,1.722657
B.F 2.1,1.028974,0.000000,2.496764,1.822662,1.776528,0.922868,1.782082
B.F 3.1,2.528423,2.496764,0.000000,2.060382,2.144976,2.493813,1.946281
B.F 4.1,1.686389,1.822662,2.060382,0.000000,1.693741,1.669384,0.250574
B.F 5.1,2.117143,1.776528,2.144976,1.693741,0.000000,2.108607,1.658396
B.F 6.1,0.334186,0.922868,2.493813,1.669384,2.108607,0.000000,1.697775
B.F 7.1,1.722657,1.782082,1.946281,0.250574,1.658396,1.697775,0.000000


In [46]:
cat_vars = ['Sex', 'Education', 'Ethnicity Race',
           'Smoke during pregnancy', 'Alcohol',
           'Post-natal multivitamins']

for var in cat_vars:
    print(f"\n----------\nVARIABLE: {var}")
    
    
    for distance_matrix, dist_name in [(wuni, "weighted UniFrac"), 
                                       (uuni, "Unweighted UniFrac"), 
                                       (bray, "Bray-Curtis")]:
        
        print("\n", dist_name.upper())
        
        #Time 0
        print("TIMEPOINT 0")
        dist = DistanceMatrix(distance_matrix.to_data_frame().loc[t0.index, t0.index], 
                      ids=t0.index)
        
        res = permanova(dist, t0, column=var)
        if res[5] < 0.05:
            print("**Significant**\n p= ", res[5])
            print("pseudo-F= ", res[4])
        else:
            print("Not significant")
        
        
        #Time 1
        print("TIMEPOINT 1")
        dist = DistanceMatrix(distance_matrix.to_data_frame().loc[t1.index, t1.index], 
                      ids=t1.index)
        
        res = permanova(dist, t1, column=var)
        if res[5] < 0.05:
            print("**Significant**\n p= ", res[5])
            print("pseudo-F= ", res[4])
        else:
            print("Not significant")
        
        #Time 2
        print("TIMEPOINT 2")
        dist = DistanceMatrix(distance_matrix.to_data_frame().loc[t2.index, t2.index], 
                      ids=t2.index)
        
        res = permanova(dist, t2, column=var)
        if res[5] < 0.05:
            print("**Significant**\n p= ", res[5])
            print("pseudo-F= ", res[4])
        else:
            print("Not significant")


----------
VARIABLE: Sex

 WEIGHTED UNIFRAC
TIMEPOINT 0
Not significant
TIMEPOINT 1
Not significant
TIMEPOINT 2
Not significant

 UNWEIGHTED UNIFRAC
TIMEPOINT 0
Not significant
TIMEPOINT 1
**Significant**
 p=  0.04
pseudo-F=  2.5160512777859774
TIMEPOINT 2
Not significant

 BRAY-CURTIS
TIMEPOINT 0
Not significant
TIMEPOINT 1
Not significant
TIMEPOINT 2
Not significant

----------
VARIABLE: Education

 WEIGHTED UNIFRAC
TIMEPOINT 0
Not significant
TIMEPOINT 1
Not significant
TIMEPOINT 2
Not significant

 UNWEIGHTED UNIFRAC
TIMEPOINT 0
Not significant
TIMEPOINT 1
Not significant
TIMEPOINT 2
Not significant

 BRAY-CURTIS
TIMEPOINT 0
Not significant
TIMEPOINT 1
Not significant
TIMEPOINT 2
Not significant

----------
VARIABLE: Ethnicity Race

 WEIGHTED UNIFRAC
TIMEPOINT 0
Not significant
TIMEPOINT 1
**Significant**
 p=  0.034
pseudo-F=  2.7619777612836804
TIMEPOINT 2
Not significant

 UNWEIGHTED UNIFRAC
TIMEPOINT 0
Not significant
TIMEPOINT 1
Not significant
TIMEPOINT 2
Not significant

 BR

In [50]:
print("PERMANOVA RESULTS")

for var in ['Length', 'Birth Weight', 'Gestational Age', 'Mothers Height', 
            'Mothers Weight']:
    print("\n----------------")
    
    for distance_matrix, dist_name in [(wuni, "weighted UniFrac"), 
                                       (uuni, "Unweighted UniFrac"), 
                                       (bray, "Bray-Curtis")]:
        for timepoint in [0,1,2]:

            print("\n", var ," ", dist_name.upper(), f" Timepoint {timepoint}")


            #initialize empty distance matrix
            dat = metadata.loc[metadata["Timepoint"]==timepoint, var].dropna()
            dim = dat.shape[0]
            distm = np.empty((dim, dim))

            #fill distance matrix
            for i in range(dim):
                for j in range(dim):
                    distm[i,j] = np.abs(dat.iloc[i]-dat.iloc[j])

            distm = DistanceMatrix(distm, ids=dat.index)

            #filter where we don't have data
            microbiome_distm = DistanceMatrix(uuni.to_data_frame().loc[dat.index,dat.index], 
                                      ids=dat.index)
            #do the mantel
            man = mantel(microbiome_distm, 
                        distm,
                        method="spearman")

            
            if man[1] > 0.05:
                print("Not Significant")
            else:
                print("corr coef: ", man[0])
                print("p value: ", man[1])

PERMANOVA RESULTS

----------------

 Length   WEIGHTED UNIFRAC  Timepoint 0
Not Significant

 Length   WEIGHTED UNIFRAC  Timepoint 1
Not Significant

 Length   WEIGHTED UNIFRAC  Timepoint 2
Not Significant

 Length   UNWEIGHTED UNIFRAC  Timepoint 0
Not Significant

 Length   UNWEIGHTED UNIFRAC  Timepoint 1
Not Significant

 Length   UNWEIGHTED UNIFRAC  Timepoint 2
Not Significant

 Length   BRAY-CURTIS  Timepoint 0
Not Significant

 Length   BRAY-CURTIS  Timepoint 1
Not Significant

 Length   BRAY-CURTIS  Timepoint 2
Not Significant

----------------

 Birth Weight   WEIGHTED UNIFRAC  Timepoint 0
Not Significant

 Birth Weight   WEIGHTED UNIFRAC  Timepoint 1
Not Significant

 Birth Weight   WEIGHTED UNIFRAC  Timepoint 2
Not Significant

 Birth Weight   UNWEIGHTED UNIFRAC  Timepoint 0
Not Significant

 Birth Weight   UNWEIGHTED UNIFRAC  Timepoint 1
Not Significant

 Birth Weight   UNWEIGHTED UNIFRAC  Timepoint 2
Not Significant

 Birth Weight   BRAY-CURTIS  Timepoint 0
Not Significant


In [54]:
var = 'Timepoint'
        
for distance_matrix, dist_name in [(wuni, "weighted UniFrac"), 
                                   (uuni, "Unweighted UniFrac"), 
                                   (bray, "Bray-Curtis")]:

    print("\n", var ," ", dist_name.upper())


    #initialize empty distance matrix
    dat = metadata[var].dropna()
    dim = dat.shape[0]
    distm = np.empty((dim, dim))

    #fill distance matrix
    for i in range(dim):
        for j in range(dim):
            distm[i,j] = np.abs(dat.iloc[i]-dat.iloc[j])

    distm = DistanceMatrix(distm, ids=dat.index)

    #filter where we don't have data
    microbiome_distm = DistanceMatrix(uuni.to_data_frame().loc[dat.index,dat.index], 
                              ids=dat.index)
    #do the mantel
    man = mantel(microbiome_distm, 
                distm,
                method="spearman")


    if man[1] > 0.05:
        print("Not Significant")
    else:
        print("corr coef: ", man[0])
        print("p value: ", man[1])


 Timepoint   WEIGHTED UNIFRAC
Not Significant

 Timepoint   UNWEIGHTED UNIFRAC
Not Significant

 Timepoint   BRAY-CURTIS
Not Significant
